In [1]:
from pymongo import MongoClient
import pandas as pd

dist_file_info = MongoClient("127.0.0.1", 27017)["radar"]["distribution_file_info"]
df = pd.DataFrame(dist_file_info.find({}, projection={"_id": 0}))
print(len(df))

7685017


In [2]:
df["packagetype"].value_counts()

bdist_wheel      3805987
sdist            3735365
bdist_egg         119540
bdist_wininst      17024
bdist_dumb          5963
bdist_msi            584
bdist_rpm            509
bdist_dmg             45
Name: packagetype, dtype: int64

In [18]:
sdist_packages = set(df[df["packagetype"] == "sdist"]["name"].unique())
bdist_packages = set(
    df[df["packagetype"].isin(["bdist_wheel", "bdist_egg"])]["name"].unique()
)
both_packages = sdist_packages.intersection(bdist_packages)
print(
    f"Number of sdist packages: {len(sdist_packages)}, Number of packages with bsit: {len(bdist_packages)}"
)
print(
    f"intersection: {len(both_packages)}, sdist-only packages: {len(sdist_packages) - len(both_packages)}, bdist-only packages: {len(bdist_packages) - len(both_packages)}"
)

Number of sdist packages: 377879, Number of packages with bsit: 283886
intersection: 246094, sdist-only packages: 131785, bdist-only packages: 37792


In [19]:
sdist_releases = df[df["packagetype"] == "sdist"][["name", "version"]].drop_duplicates()
bdist_releases = df[df["packagetype"].isin(["bdist_wheel", "bdist_egg"])][
    ["name", "version"]
].drop_duplicates()
both_releases = sdist_releases.merge(bdist_releases)
print(
    f"Number of sdist releases: {len(sdist_releases)}, Number of bdist releases: {len(bdist_releases)}"
)
print(
    f"intersection: {len(both_releases)}, sdist-only releases: {len(sdist_releases) - len(both_releases)}, bdist-only releases: {len(bdist_releases) - len(both_releases)}"
)

Number of sdist releases: 3720506, Number of bdist releases: 2892008
intersection: 2419437, sdist-only releases: 1301069, bdist-only releases: 472571


In [33]:
from packaging.version import Version


def not_prerelease(x):
    try:
        return not Version(x).is_prerelease
    except:
        return False


sampled_release = both_releases.groupby("name").sample(n=1, random_state=1)
print(
    len(sampled_release),
    len(sampled_release[sampled_release["version"].apply(not_prerelease)]),
)

243545 230852


In [36]:
sampled_release_info = sampled_release.merge(df)
print(len(sampled_release_info))
sampled_release_info.to_csv("../data/selected_dist.csv", index=False)

548224
